In [88]:
from Bio import SeqIO
import urllib.parse as up
import os
import psycopg2
import re
from Bio import Entrez
from collections import Counter
from IAP_seqs import Seq

Ligar Base de Dados

In [89]:
import mysql.connector as SQLC   #ligação Bruno

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()


# Ficheiro GB Web

Permite através do identificador unico (accession)  obter todos os dados acerca do mesmo sem necessidade de um ficheiro local

Para isto foram utilizados divervos submódulos pertencentes ao BioPython entre os quais o Entrez que nos permitiu aceder á base do NCBI e aceder à informação relativa ao nosso organimo, foi também utilizado o SeqIO que nos permitiu ler a informação recolhida do NCBI

In [90]:
id_genbank_input = input("ID NCBI")
handle = Entrez.efetch(db='nucleotide',retmode='txt',id=id_genbank_input, rettype = "gb")
record = SeqIO.read(handle, "gb")

# Ficheiro GB Local

Usado inicialmente na contrução do código

In [91]:
# # file = "/home/sa_bruno/Documentos/GitHub/Projeto_LB/ARL15/ARL15.gb"                
# file = "Teste.gb"
# record = SeqIO.read(file, "genbank") 

# Obter todas as variáveis / Povoar Base de Dados 

# Tabela Locus

Devido à utilização do objeto SeqRecord originado através do SeqIO conseguimos diretamente extrair a informação necessária para Povoar esta tabela

In [92]:
# Locus 
ID = record.name
Size = len(record.seq)
Molecular_type = record.annotations["molecule_type"] + " " + record.annotations["topology"]
Genbank_division = record.annotations["data_file_division"] 
Modification_date = record.annotations["date"]

In [93]:
print(ID)
print(Size)
print(Molecular_type)
print(Genbank_division)
print(Modification_date)

NC_038931
13669
DNA linear
VRL
07-JAN-2023


In [94]:
try:
        cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

except:
        print("Erro Povoação Locus")

# Tabela Genbank

Utilizamos novamente o objeto SeqRecord para retirar diretamente a informação; neste caso foi necessário um tratamento especial da coluna KeyWords visto que esta pode possuir diversos items (lista) logo é necessário juntar os mesmos e tornar em String para posteriormente ensirir na nossa tabela


In [95]:
# Genbank
Organism = record.annotations["organism"]
Accession = record.annotations["accessions"][0]
Definition = record.description
Keywords = ""
ID_version_seq = record.id
for c in record.annotations["keywords"]:
    Keywords += c + ", "
Keywords = Keywords[0: (len(Keywords) -2)]

Sequence = str(record.seq)

In [96]:
print(ID_version_seq)
print(Organism)
print(Accession)
print(Definition)
print(Keywords)
print(Sequence)

NC_038931.1
Thermoproteus tenax virus 1
NC_038931
Thermoproteus tenax virus 1 (TTV1) genome
RefSeq, coat protein, genome, promoter, structural protein, unidentified reading frame
TGTGTGATGTGAGCGATACATTCACACAGAACAGCTATGACCATGATTACCAATTCCCAGAATGTTTAAGTGAAATCGAAGCCACAAGAAAGGGATATAAGGAGTTCGTAGAATGCGTATGCGAACAGTTACTGGAGAAGATAGCTGAAGAGTATTGCAAAGGTATGGGTGTGATTGGAAGTCAAATGGAAGAGTGCAAGGATAAGTATATTGAGACTCATAGGCACGTATTTCCTTGTGAGGACGTTGTGAGACAATGGTTAGTAGAGATGGGAGAGAAGCTGAAGGGCCATGGTAGACTTTCCAGACATTGAGGTTGAAAGTATACCTGAGGAGATTATGGAAGAGGTAGGCGAATATATCGATATCTTCATTCAACTAACTGAAATTGCTGAGATGGAAAGAGACGTTATCGTGGAGGAACAATTGAAAGATGAAGCAAAACAATTCGAAGAATATGAAGATTTCTGGTTTGACGTATACACATGACGAGATACAATTACATTAGGAATCTAAGAATAATCAGAGGAGAGGAAATCAAAGTAGGTGTGAGTAGCGATAGAATTCCAATGATAGAGATTGAGCCAGGACACAATGACCCAGAAAGAATGTGTGTAATGTTCACGGCTATGGAATATAGTCATATACCATATCAAGCAGATAATCTAGACGAAGTATATGACTCATGTAAAATTTTGACAAGCAAGATTACTACTTACATAAATGTAGCAATAGGATTTGGCAAGAATTTGGGGCTCAAATTAAGCGAGATGATGTTGGTGACTAGAGATAAGTCGAAAAGAGAATTCAAAGTTGGATTA

In [97]:
try:
        cur.execute(f"""
                 INSERT INTO Genbank (ID_version_seq, Organism, Accession, Definition, Keywords, Id_locus)    
                VALUES (%s, %s, %s, %s, %s, %s)
                """,
                        (ID_version_seq, Organism, Accession, Definition, Keywords,ID ))
except:
        print("Erro Povoação Genbank")

In [98]:
# cur.execute(f"""
#                  INSERT INTO Genbank (ID_version_seq, Organism, Accession, Definition, Keywords, Id_locus)    
#                 VALUES (%s, %s, %s, %s, %s, %s)
#                 """,
#                         (ID_version_seq, Organism, Accession, Definition, Keywords, ID ))

# Tabela Sequences  / Parte para Algoritmos

Esta Tabela destina-se para o uso da parte relativa a Algoritmos, visto a entrega deste trabalho ser muito posterior apenas possuimos um pequeno excerto do mesmo de modo a demonstrar a interação.

Neste tabela importamos uma Classe (Seq) desenvolvida por nós no ambito da cadeira de Algoritmos e pegando na sequencia préviamente obtida conseguimos analisar a quantidade de nucleotidos associados á mesma através da utilizaçao da nossa classe 

In [99]:
alg_seq = Seq(Sequence).count_nucleotides()

Count_A = alg_seq[0]
Count_C = alg_seq[1]
Count_G = alg_seq[2]
Count_T = alg_seq[3]

In [100]:
print(ID)
print(Sequence)
print(Count_A)
print(Count_C)
print(Count_G)
print(Count_T)

NC_038931
TGTGTGATGTGAGCGATACATTCACACAGAACAGCTATGACCATGATTACCAATTCCCAGAATGTTTAAGTGAAATCGAAGCCACAAGAAAGGGATATAAGGAGTTCGTAGAATGCGTATGCGAACAGTTACTGGAGAAGATAGCTGAAGAGTATTGCAAAGGTATGGGTGTGATTGGAAGTCAAATGGAAGAGTGCAAGGATAAGTATATTGAGACTCATAGGCACGTATTTCCTTGTGAGGACGTTGTGAGACAATGGTTAGTAGAGATGGGAGAGAAGCTGAAGGGCCATGGTAGACTTTCCAGACATTGAGGTTGAAAGTATACCTGAGGAGATTATGGAAGAGGTAGGCGAATATATCGATATCTTCATTCAACTAACTGAAATTGCTGAGATGGAAAGAGACGTTATCGTGGAGGAACAATTGAAAGATGAAGCAAAACAATTCGAAGAATATGAAGATTTCTGGTTTGACGTATACACATGACGAGATACAATTACATTAGGAATCTAAGAATAATCAGAGGAGAGGAAATCAAAGTAGGTGTGAGTAGCGATAGAATTCCAATGATAGAGATTGAGCCAGGACACAATGACCCAGAAAGAATGTGTGTAATGTTCACGGCTATGGAATATAGTCATATACCATATCAAGCAGATAATCTAGACGAAGTATATGACTCATGTAAAATTTTGACAAGCAAGATTACTACTTACATAAATGTAGCAATAGGATTTGGCAAGAATTTGGGGCTCAAATTAAGCGAGATGATGTTGGTGACTAGAGATAAGTCGAAAAGAGAATTCAAAGTTGGATTAGAATTGGGAATAAAATATGCATATAATTGGACCTGTAAGTTTCAGTAGGACGGTATTCCTAGATTTCTACAATTATTTATTACAAAGTGGTCTTTATGAAACGCTAAGAATAGATAGAAAGTACATTGAATATACATTGAAAAATAAATATGCTACACTAAGATTATAT

In [101]:
try:

        cur.execute("""
                INSERT INTO Sequences ( Count_A, Count_C, Count_T, Count_G, Sequence, ID_version_seq)    
                VALUES (%s, %s, %s, %s, %s,%s)
                """,
                        (Count_A, Count_C, Count_T, Count_G, Sequence,ID_version_seq))   

except:
        print("Erro Povoação Sequences")

In [102]:
# cur.execute("""
#                 INSERT INTO Sequences ( Count_A, Count_C, Count_T, Count_G,Sequence, ID_version_seq)    
#                 VALUES (%s, %s, %s, %s, %s,%s)
#                 """,
#                         ( Count_A, Count_C, Count_T, Count_G, Sequence,ID_version_seq))   

# Tabela Features

Visto existir uma enorme quantidade de diferentes tipos de features tomamos a decisão de apenas contabilizar na nossa tabela os tipos abaixo indicados, visto estes serem os mais relevantes a nivél biológico.

In [103]:
temp_l = []
for c in range(len(record.features)):
    temp_l.append(record.features[c].type)

y = Counter(temp_l)
Gene = y["gene"]
# Regulatory = 
#(ID, CDS, Gene, Regulatory, exons, poli_A-site, misc_feature, mRNA))      # acabar esta parte dps  easy
# so contar apartir daqui 
  # Counter({'gene': 5125, 'CDS': 4997, 'tRNA': 105, 'rRNA': 22, 'source': 1, 'tmRNA': 1})
CDS_count = y["CDS"]
Regulatory = y["regulatory"]
exons = y["exons"]
poli_A_site = y["poli_A-site"]                    #  poli_A-site
misc_feature = y["misc_feature"]
mRNA = y["mRNA"]
Location_span = str(record.features[0].location)

In [104]:
print(ID)
print(CDS_count)
print(Gene)
print(Regulatory)
print(poli_A_site)
print(misc_feature)
print(mRNA)
print(Location_span)

NC_038931
37
37
1
0
0
0
[0:13669](+)


In [105]:
try:
        cur.execute("""
        INSERT INTO Features (ID_version_genbank, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID_version_seq, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 
except:
        print("Erro Features")

In [106]:
# cur.execute("""
#         INSERT INTO Features (ID_version_genbank, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
#         """,
#                 (ID_version_seq, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 

# Tabela CDS

Devido á elevada importância dos CDS (tipo de feature) decidimos criar uma tabela especificamente para os mesmos. Visto poder haver a existência de um elevado numero de Cds em cada ficheiro foi necessário trabalharmos com listas de forma a organizar melhor a informação.

In [107]:
# usar para dps
CDS_list = []

ID_cds = []
Translacion_seq = []
Localization = []
ID_protein = []
Protein = []
for c in range(len(record.features)):
    if record.features[c].type == "CDS":
        CDS_list.append(record.features[c])

# Translacion_seq
for c in range(len(CDS_list)):
    ID_cds.append(CDS_list[c])
    Translacion_seq.append(CDS_list[c].qualifiers["translation"])
    Localization.append(CDS_list[c].location)
    ID_protein.append(CDS_list[c].qualifiers["protein_id"])
    try:
        Protein.append(CDS_list[c].qualifiers["product"])
    except:
        Protein.append("------")
    


In [108]:
for c in range(len(ID_cds)):
    print(ID_cds[c])
    print()
    print(Translacion_seq[c])
    print()
    print(Localization[c])
    print()
    print(ID_protein[c])
    print()
    print(Protein[c])
    print()
    print()


type: CDS
location: [291:489](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['UniProtKB/Swiss-Prot:P19276', 'GeneID:37619647']
    Key: locus_tag, Value: ['D1T48_gp01']
    Key: note, Value: ['URF (65 AA)']
    Key: product, Value: ['hypothetical protein']
    Key: protein_id, Value: ['YP_009509117.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MVDFPDIEVESIPEEIMEEVGEYIDIFIQLTEIAEMERDVIVEEQLKDEAKQFEEYEDFWFDVYT']


['MVDFPDIEVESIPEEIMEEVGEYIDIFIQLTEIAEMERDVIVEEQLKDEAKQFEEYEDFWFDVYT']

[291:489](+)

['YP_009509117.1']

['hypothetical protein']


type: CDS
location: [485:869](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['UniProtKB/Swiss-Prot:P19277', 'GeneID:37619611']
    Key: locus_tag, Value: ['D1T48_gp02']
    Key: note, Value: ['URF (127 AA)']
    Key: product, Value: ['hypothetical protein']
    Key: protein_id, Value: ['YP_009509118.1']
    Key: transl_table, Value: ['11']
    Key: translation, Valu

In [109]:
num_list = 0                            # cdc ID que falta ver
for c in range(len(ID_cds)):
        cur.execute("""
                INSERT INTO CDS (ID_protein_cds, Translacion_seq, Localization, Protein,id_version_features)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                (str(ID_protein[num_list]), str(Translacion_seq[num_list]), str(Localization[num_list]), str(Protein[num_list]), ID_version_seq))
        num_list += 1

# Tabela Reference

Na Tabela Reference visto não haver acesso direto á informação pretendida através do SeqRecord tivémos de recorrer a expressões regulares de forma a obter a informação pretendida.

Criámos uma pequena funçao auxiliar (get_authors) que nos permitiu obter os autores e fazer o devido tratamento dos mesmos.

In [110]:
def get_authors(a):
    exis_5 = re.search(r"authors:\s+([\w\s,\.-]+)", a)          #dúvida (como é que o ciclo sabe se isto está vazio ou não)       
    if exis_5:
        Authors = []
        Authors_work = []
        y = re.search(r"authors:\s+([\w\s,\.-]+)", a)    # isto aqui é o remark
        Authors_work.append(y.group(1))
        # else:
        #     Consortium.append("------")
        help_authors = [] 
        Authors = str(Authors_work[0]).split("., ")
        for i, c in enumerate(Authors):
            f = c.split(" and ")
            help_authors.append([i, f])
        c = len(Authors)
        i = 0
        while c > 0:
            if len(help_authors[i][1]) > 1:
                Authors.remove(Authors[i])
                for o in help_authors[i][1]:
                    Authors.append(o)

            c -= 1
            i += 1
    else:
        Authors = []

    return Authors



In [111]:
# referencias  # faz parte do PubMed INfo
Pubmed = []
Title = []
Journal = []
Consortium = []
Remark = []
Authors = []
b = ""
pubmed_trash = record.annotations["references"]    # resolver isto 
c = len(pubmed_trash)
i = 0
while c > 0:
    a = str(pubmed_trash[i]).replace("\n", "§")   
    
    exist_3 = re.search(r"pubmed id:\s+(\d+)", a)
    if exist_3:
        m = re.search(r"pubmed id:\s+(\d+)", a)                 # COnfirmar PubMEd haver sempre/nao
        Pubmed.append(m.group(1))
    else:
        Pubmed.append("None")

    
    n = re.search(r"title:\s+([\w\s')(-]+)", a)
    Title.append(n.group(1))

    exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)
    if exist_2:
        j = re.search(r"journal:\s+([\w\s.()--,]+)", a)
        Journal.append(j.group(1))
    else:
        Journal.append("None")

    exis = re.search(r"consrtm:\s+([\w\s]+)", a)
    if exis:
        k = re.search(r"consrtm:\s+([\w\s]+)", a)    # isto aqui é o remark
        Consortium.append(k.group(1))
    else:
        Consortium.append("None")         # ter cuidado com isto dps
    

    exis_1 = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\s\d])", a)
    if exis_1:
        l = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\d\s:\[;\(\)\.,!?<>{}\"]+)", a)
        Remark.append(l.group(1))
    else:
        Remark.append("None") 

    Authors.append(get_authors(a))


    
    c -= 1
    i += 1

In [112]:
print(ID)
print(Pubmed)
print(Authors)
print(Title)
print(Journal)
print(Consortium)
print(Remark)

NC_038931
['2308830', '2505050', 'None', 'None']
[['Neumann,H.', 'Zillig,W.'], ['Neumann,H', 'Schwass,V', 'Eckerskorn,C.', 'Zillig,W.'], [], ['Neumann,H.']]
['The TTV1-encoded viral protein TPX', 'Identification and characterization of the genes encoding three structural proteins of the Thermoproteus tenax virus TTV1', 'Direct Submission', 'Direct Submission']
['Nucleic Acids Res. 18 (1), 195 (1990)', 'Mol. Gen. Genet. 217 (1), 105-110 (1989)', 'Submitted (24-AUG-2018) National Center for Biotechnology Information, NIH, Bethesda, MD 20894, USA', 'Submitted (28-MAR-1989) Neumann H., Max Planck Institut fuer Biochemie, Am Klopferspitz, 8033 Martinsried, F R G']
['None', 'None', 'NCBI Genome Project', 'None']
['None', 'None', 'None', 'None']


In [113]:
reference_count = 0                 
for c in Pubmed:    
        cur.execute("""
                INSERT INTO reference (Journal_ID, Pubmed, Consortium, Remark,Title)
                VALUES (%s, %s, %s, %s, %s)
                """,
                (str(Journal[reference_count]), str(Pubmed[reference_count]), str(Consortium[reference_count]), str(Remark[reference_count]), str(Title[reference_count])))
        reference_count += 1

Tabela genbank-references

In [114]:
count = 0
for c in Pubmed: 
        cur.execute("""
                INSERT INTO genbank_reference (id_version_connect, journal_connect)    
                VALUES (%s, %s)
                """,
                        (ID_version_seq, str(Journal[count]))) 
        count += 1

# Tabela Authors


In [115]:
try:
    trash_list = [str(a) for c in Authors for a in c]
    c = 0
    while len(trash_list) > c:
        cur.execute(f"""
            INSERT INTO Authors (Names)    
            VALUES (%s)
            """,
            (trash_list[c],))
        c += 1
except:
    print("Erro Authors")

Erro Authors


# Tabela reference_authors

In [116]:
count = 0
for i, c  in enumerate(Journal):
    for o in Authors[i]:
        cur.execute("""
                INSERT INTO reference_authors (reference_journal_key, authors_key)    
                VALUES (%s, %s)
                """,
                        (Journal[i], o))
        count += 1
    

# Tabela Pubmed_Info

Nesta Tabela utilizamos as referências do PubMed anteriormente obtidas através do  ficheiro GB e acedemos às mesmas de forma a retirar  informaçáo pretendida recorrendo novamente ao Entrez e a expressões regulares.

In [117]:
Abstract = []                        # mudar para cada referenias 
DOI = []
Affiliation = []
for c in Pubmed:
    if c != "None":
        Entrez.email = 'sa.bruno.2001@gmail.com'
        # handle = Entrez.efetch(db='pubmed',retmode='xml',id="15496913")
        handle = Entrez.efetch(db='pubmed',retmode='xml',id=c)
        resultado = Entrez.read(handle)
        q = str(resultado["PubmedArticle"][0])
        
        if re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q):
            o = re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q)
            Abstract.append(o.group(1))
        else:
            Abstract.append("None")
        
        if re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q):
            p = re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q)
            DOI.append(p.group(1))
        else:
            DOI.append("None")

        if re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q):
            t = re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q)
            Affiliation.append(t.group(1))
        else:
            Affiliation.append("None")
    if c == "None":
            Abstract.append("None")
            DOI.append("None")
            Affiliation.append("None")



In [118]:
print(Pubmed)
print(Title)
print(Abstract)
print(DOI)


['2308830', '2505050', 'None', 'None']
['The TTV1-encoded viral protein TPX', 'Identification and characterization of the genes encoding three structural proteins of the Thermoproteus tenax virus TTV1', 'Direct Submission', 'Direct Submission']
['None', "Three structural proteins, TP1, TP2 and TP3, of the virus TTV1 of the thermophilic archaebacterium Thermoproteus tenax strain Kra1 were mapped within the viral genome by locating the amino-terminal amino acid sequences of these proteins in the TTV1 DNA sequence. The derived amino acid sequences comprise 113, 139 and 160 amino acid residues, respectively. All three proteins are hydrophobic. The three genes are not linked, but transcribed in the same direction. No Shine-Dalgarno sequences are found in the vicinity of the initiation codons of these three genes. By Northern analysis, four mRNAs of 0.5 kb, 0.8 kb, 1.1 kb and 1.8 kb in size were found to be encoded in the region and in the vicinity of the genes, the shortest one (t1) encodin

In [119]:
pubmed_count= 0
for c in Pubmed :
        if c != "None":
                cur.execute("""
                        INSERT INTO Pubmed_Info (ID_journal, Title, Abstract, DOI, Affiliation)   
                        VALUES (%s, %s, %s, %s, %s)
                        """,
                        (Journal[pubmed_count], Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count], Affiliation[pubmed_count]))  
                pubmed_count += 1 
        else:
                pubmed_count += 1